In [1]:
import os

In [2]:
pwd

'/config/workspace/red_wine_quality/research'

In [3]:
os.chdir('../')

In [4]:
pwd

'/config/workspace/red_wine_quality'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    unzip_data_path: Path
    STATUS_FILE: str
    all_schema: dict

In [6]:
## configuration manager

from wine_quality.utils.common import read_yaml, create_directories
from wine_quality.constants import *

class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, schema_filepath=SCHEME_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.schema = read_yaml(schema_filepath)
        self.params = read_yaml(params_filepath)
    
        create_directories([self.config.artifacts_root])

    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir = config.root_dir,
            unzip_data_path = config.unzip_data_path,
            STATUS_FILE = config.STATUS_FILE,
            all_schema = schema

        )

        return data_validation_config

In [11]:
import os
from wine_quality import logger
import pandas as pd

class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config

    def validate_all_columns(self):
        try:
            validation_status = None
            data = pd.read_csv(self.config.unzip_data_path)
            all_cols = list(data.columns)
            all_schema = self.config.all_schema.keys()

            for col in all_cols:
                if col not in all_schema:
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"validation status: {validation_status}")
                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"validation status: {validation_status}")

            return validation_status
        except Exception as e:
            raise e

In [12]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config = data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e

[2024-01-05 02:17:01,717: INFO: common: yaml file: config/config.yaml loaded successfully.]
[2024-01-05 02:17:01,720: INFO: common: yaml file: schema.yaml loaded successfully.]
[2024-01-05 02:17:01,723: INFO: common: yaml file: params.yaml loaded successfully.]
[2024-01-05 02:17:01,723: INFO: common: created directory at: artifacts]
[2024-01-05 02:17:01,724: INFO: common: created directory at: artifacts/data_validation]
